In [1]:
import AlgoData as prp
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import tensorflow
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

In [ ]:
df = prp.ReadTrainData('train.csv')

In [ ]:
X = df.drop(['Price', 'price_log_transform', 'index'], axis=1)
y = df['Price']

In [ ]:
#Randomized Search CV for searching the best parameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10]}


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()

In [ ]:
rf_random_model = RandomizedSearchCV(estimator = rf,
                                     param_distributions = random_grid,
                                    scoring='neg_mean_squared_error',
                                    n_iter = 10,
                                    cv = 5,
                                    verbose=2,
                                    random_state=42,
                                    n_jobs = 1)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
rf_random_model.fit(Xtrain,ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   9.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   7.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   7.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   7.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   8.0s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=  12.2s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=  11.6s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [ ]:
f = open('beta_RF_final_0_0_4.sav', 'wb')
prp.pickle.dump(rf_random_model, f)
f.close()

In [3]:
import pickle
filename = 'beta_RF_final_0_0_4.bin'
model = pickle.load(open(filename, 'rb'))

EOFError: Ran out of input

In [ ]:
type(model)

sklearn.model_selection._search.RandomizedSearchCV

In [ ]:
y_pred = np.exp(model.predict(Xtest))

In [ ]:
data = {
    'YTrue': ytest,
    'YPred' : y_pred
}
 
# Create the pandas DataFrame
df = prp.p.DataFrame(data=data)
 
# print dataframe.
df

,YTrue,YPred
1770,20700,17925.595162
639,157,477.359932
15934,9408,14631.553354
10197,14113,11154.482126
936,10349,7452.501504
...,...,...
7972,1568,5574.503597
10314,4861,5328.603661
17442,21326,9592.036105
5433,18503,17317.753984


In [ ]:
MAE = prp.mean_absolute_error(ytest, y_pred)
MAPE = prp.mean_absolute_percentage_error(ytest, y_pred)

In [ ]:
MAPE

5.941345304679839